#### Data location
#### https://valuation.property.nsw.gov.au/embed/propertySalesInformation
#### http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181001.zip 
#### http://www.valuergeneral.nsw.gov.au/__psi/yearly/2017.zip

### Metadata
http://www.valuergeneral.nsw.gov.au/__data/assets/pdf_file/0015/216402/Current_Property_Sales_Data_File_Format_2001_to_Current.pdf
http://www.valuergeneral.nsw.gov.au/__data/assets/pdf_file/0016/216403/Property_Sales_Data_File_-_Data_Elements_V3.pdf

In [1]:
import zipfile
import pandas as pd
#import matplotlib.pyplot as plt
from datetime import datetime
import collections
import requests
import os
import requests
import io
import zipfile
import datetime
import time 
import glob
import csv


import glob
import csv
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import collections
import requests
import os
import io
import zipfile
import datetime
import time 

#from datetime import datetime

#### Weekly load

In [52]:
def load_sales_data(file_path, folder_level=-2):
    dat_files = []
    for root, dirs, files in os.walk(file_path):
        for file in files:
            if file.endswith(".DAT"):
                dat_files.append(os.path.join(root, file))
    print(dat_files)
    #print("Loading file from - {0}/**/*.DAT".format(file_path))
    #dat_files = [f for f in glob.iglob("{0}/*.DAT".format(file_path), recursive=True)]
    foldername = (file_path.split("/")[folder_level])
    print("Loading sales data for the date - " + foldername)
    try:
        os.remove("../data/sales/A_str/{0}.csv".format(foldername))
        os.remove("../data/sales/B_str/{0}.csv".format(foldername))
        os.remove("../data/sales/C_str/{0}.csv".format(foldername))
        os.remove("../data/sales/D_str/{0}.csv".format(foldername))
        os.remove("../data/sales/Z_str/{0}.csv".format(foldername))
    except OSError:
        pass
    for i in dat_files:
        A_str = ""
        B_str = ""
        C_str = ""
        D_str = ""
        Z_str = ""
        with open(i) as f:
            lis=[line.split() for line in f]
            for i in lis:
                if ("".join(i)).split(";")[0] == 'A':
                    A_str =  A_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'B':
                    B_str = B_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'C':
                    C_str = C_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'D':
                    D_str = D_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'Z':
                    Z_str = Z_str + ";".join(("".join(i)).split(";")) + "\n"
        with open("../data/sales/A_str/{0}.csv".format(foldername), "a") as a_myfile:
            a_myfile.write(A_str)
        with open("../data/sales/B_str/{0}.csv".format(foldername), "a") as b_myfile:
            b_myfile.write(B_str)
        with open("../data/sales/C_str/{0}.csv".format(foldername), "a") as c_myfile:
            c_myfile.write(C_str)
        with open("../data/sales/D_str/{0}.csv".format(foldername), "a") as d_myfile:
            d_myfile.write(D_str)
        with open("../data/sales/Z_str/{0}.csv".format(foldername), "a") as z_myfile:
            z_myfile.write(Z_str)

def download_extract_zip(url):
    response = requests.get(url)
    if (response.status_code==200):
        with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
            path2download = "../data/temp/{0}/".format((url.split("/")[-1]).replace(".zip",""))
            thezip.extractall(path2download)
            print("Files are downloaded successfully in {0} !!!".format(path2download))
            load_sales_data(path2download)
            #return path2download
    else:
        print("No data found")
#download_extract_zip(url)                



def property_price_sq_meter(row):
    if row['Area_Type'] == 'M':
        return  row['Purchase_Price'] / row['Area']
    if row['Area_Type'] =='H':
        return  row['Purchase_Price'] / (row['Area'] * 10000)

def sale_datamart(year):
    # Data Validation
    col_header =['Record_Type',
    'District_Code',
    'Property_Id',
    'Sale_Counter',
    'Download_Date_Time',
    'Property_Name',
    'Property_Unit_Number',
    'Property_House_Number',
    'Property_Street_Name',
    'Property_Locality',
    'Property_Post_Code',
    'Area',
    'Area_Type',
    'Contract_Date',
    'Settlement_Date',
    'Purchase_Price',
    'Zoning',
    'Nature_of_Property',
    'Primary_Purpose',
    'Strata_Lot_Number',
    'Component_code',
    'Sale_Code',
    'Percentage_Interest_of_Sale',
    'Dealing_Number',
    'Empty']
    
    
    print("Loading datamart .. "  + '../data/sales/B_str/{}.csv'.format(year))
    lst_pds = [pd.read_csv(f, 
                           sep=';', 
                           error_bad_lines=False, 
                           quoting=csv.QUOTE_NONE, 
                           header=None ,  
                           encoding='utf-8',
                       index_col=None
                      ) for f in glob.glob('../data/sales/B_str/{}.csv'.format(year))]
    df_sale = pd.concat(lst_pds)
    df_sale.columns = col_header
    
    # Add Downloaded_Date from 
    df_sale['Download_Date'] = pd.to_datetime(df_sale['Download_Date_Time'].str[:8])
    df_sale['Settlement_Date_f'] = pd.to_datetime(df_sale['Settlement_Date'],format='%Y%m%d') 

    #Deduplicate 
    df_sale_dedup = df_sale.drop_duplicates(subset=[i for i in df_sale.columns], keep=False)
    
    df_sale_dedup['Property_Price_Sq_Meter'] = df_sale_dedup.apply(property_price_sq_meter, axis=1)
    # Build Data mart
    cols_lst     = ['Settlement_Date_f','Property_Locality', "Property_Post_Code"]
    cols_lst_key = ["Property_Price_Sq_Meter"]
    
    agg_lst      = ['mean','count', 'min', 'max', 'median','std']
    
    df_time_ser_price = df_sale_dedup[(df_sale_dedup.Nature_of_Property == 'R') & 
                                      (df_sale_dedup.Area_Type.notnull()) & 
                                      (df_sale_dedup.Property_Unit_Number.isnull())]\
                                            [cols_lst + cols_lst_key ]\
                                                .groupby(cols_lst)\
                                                .agg(agg_lst)
    
    df_time_ser_price =  df_time_ser_price.reset_index()
    df_time_ser_price.columns = cols_lst + [grp + '_' + agg for grp in cols_lst_key for agg in agg_lst]
    print(len(df_time_ser_price)) 
    # Export to csv file 
    df_time_ser_price.to_csv('../data/sales/sales_{}_data_mart.csv'.format(year.replace("*","")),index=False)
    
    
def sale_datamart(year):
    # Data Validation
    col_header =['Record_Type',
    'District_Code',
    'Property_Id',
    'Sale_Counter',
    'Download_Date_Time',
    'Property_Name',
    'Property_Unit_Number',
    'Property_House_Number',
    'Property_Street_Name',
    'Property_Locality',
    'Property_Post_Code',
    'Area',
    'Area_Type',
    'Contract_Date',
    'Settlement_Date',
    'Purchase_Price',
    'Zoning',
    'Nature_of_Property',
    'Primary_Purpose',
    'Strata_Lot_Number',
    'Component_code',
    'Sale_Code',
    'Percentage_Interest_of_Sale',
    'Dealing_Number',
    'Empty']
    
    
    print("Loading datamart .. "  + '../data/sales/B_str/{}.csv'.format(year))
    lst_pds = [pd.read_csv(f, 
                           sep=';', 
                           error_bad_lines=False, 
                           quoting=csv.QUOTE_NONE, 
                           header=None ,  
                           encoding='utf-8',
                       index_col=None
                      ) for f in glob.glob('../data/sales/B_str/{}.csv'.format(year))]
    df_sale = pd.concat(lst_pds)
    df_sale.columns = col_header
    
    # Add Downloaded_Date from 
    df_sale['Download_Date'] = pd.to_datetime(df_sale['Download_Date_Time'].str[:8])
    df_sale['Settlement_Date_f'] = pd.to_datetime(df_sale['Settlement_Date'],format='%Y%m%d') 

    #Deduplicate 
    df_sale_dedup = df_sale.drop_duplicates(subset=[i for i in df_sale.columns], keep=False)
    df_sale_dedup['Property_Price_Sq_Meter'] = df_sale_dedup.apply(property_price_sq_meter, axis=1)
    
    
    
    cols_lst     = ['Settlement_Date_f','Property_Locality', "Property_Post_Code"]
    cols_lst_key = ["Purchase_Price","Property_Price_Sq_Meter"]
    agg_lst      = ['mean','count', 'min', 'max', 'median','std']
    
    # filters 
    fltr_residance   = df_sale_dedup.Nature_of_Property == 'R'
    fltr_area_type   = df_sale_dedup.Area_Type.notnull()
    fltr_unit_number_h = df_sale_dedup.Property_Unit_Number.isnull() # for house
    fltr_unit_number_u = df_sale_dedup.Property_Unit_Number.notnull() # for units
    
    # House 
    df_sale_dedup_agg_h = df_sale_dedup[fltr_residance & 
                                        fltr_area_type & 
                                        fltr_unit_number_h]\
                        [cols_lst + cols_lst_key ]\
                        .groupby(cols_lst)\
                        .agg(agg_lst)
    
    df_sale_dedup_agg_h =  df_sale_dedup_agg_h.reset_index()
    df_sale_dedup_agg_h.columns = cols_lst + [grp + '_' + agg for grp in cols_lst_key for agg in agg_lst]
    print(len(df_sale_dedup_agg_h)) 
    df_sale_dedup_agg_h.insert(loc=0, column='category', value="house")
    
    
    # Units 
    df_sale_dedup_agg_u = df_sale_dedup[fltr_residance & 
                                        fltr_area_type & 
                                        fltr_unit_number_u]\
                        [cols_lst + cols_lst_key ]\
                        .groupby(cols_lst)\
                        .agg(agg_lst)
    
    df_sale_dedup_agg_u =  df_sale_dedup_agg_u.reset_index()
    df_sale_dedup_agg_u.columns = cols_lst + [grp + '_' + agg for grp in cols_lst_key for agg in agg_lst]
    print(len(df_sale_dedup_agg_u)) 
    df_sale_dedup_agg_u.insert(loc=0, column='category', value="unit")
    
    
    
    # overall 
    df_sale_dedup_agg_all = df_sale_dedup[fltr_residance & 
                                        fltr_area_type ]\
                        [cols_lst + cols_lst_key ]\
                        .groupby(cols_lst)\
                        .agg(agg_lst)
    
    df_sale_dedup_agg_all =  df_sale_dedup_agg_all.reset_index()
    df_sale_dedup_agg_all.columns = cols_lst + [grp + '_' + agg for grp in cols_lst_key for agg in agg_lst]
    print(len(df_sale_dedup_agg_all)) 
    df_sale_dedup_agg_all.insert(loc=0, column='category', value="over_all")
    
    # Concat 3 dataframes
    df_sales_all = pd.concat([df_sale_dedup_agg_h, df_sale_dedup_agg_u, df_sale_dedup_agg_all], ignore_index=True)
        
    
    # Export to csv file 
    print(len(df_sales_all)) 
    df_sales_all.to_csv('../data/sales/sales_{}_data_mart.csv'.format(year.replace("*","")),index=False)    

# Weekly load 

In [42]:
days_to_load = 20
ts = time.time()
for i in range(0, days_to_load):
    a = str(datetime.date.today() - datetime.timedelta(i)).replace("-","")
    url = 'http://www.valuergeneral.nsw.gov.au/__psi/weekly/{0}.zip'.format(a)
    print(url)
    download_extract_zip(url)
te = time.time()
print(te-ts)
#185.92472124099731

http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190219.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190218.zip
Files are downloaded successfully in ../data/temp/20190218/ !!!
Loading sales data for the date - 20190218
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190217.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190216.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190215.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190214.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190213.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190212.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190211.zip
Files are downloaded successfully in ../data/temp/20190211/ !!!
Loading sales data for the date - 20190211
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190210.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20190

In [43]:
sale_datamart("2019*")

Loading datamart .. ../data/sales/B_str/2019*.csv


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4085
612
4589
9286


In [ ]:
7478

### History Load

In [ ]:
ts = time.time()
for yr in range(2017,2018):
    url = 'http://www.valuergeneral.nsw.gov.au/__psi/yearly/{0}.zip'.format(yr)
    print(url)
    download_extract_zip(url)
    
    dat_files = [f for f in glob.glob("../data/temp/{0}/*.zip".format(yr))]
    for file in dat_files:
        with zipfile.ZipFile(file) as thezip:
            thezip.extractall("../data/temp/{0}/".format(yr))
    load_sales_data("../data/temp/{0}/".format(yr))
    sale_datamart("{}".format(yr))

    
    

http://www.valuergeneral.nsw.gov.au/__psi/yearly/2017.zip
Files are downloaded successfully in ../data/temp/2017/ !!!
['../data/temp/2017/052_SALES_DATA_NNME_29052017.DAT', '../data/temp/2017/234_SALES_DATA_NNME_09012017.DAT', '../data/temp/2017/066_SALES_DATA_NNME_03072017.DAT', '../data/temp/2017/219_SALES_DATA_NNME_08052017.DAT', '../data/temp/2017/109_SALES_DATA_NNME_16102017.DAT', '../data/temp/2017/043_SALES_DATA_NNME_09102017.DAT', '../data/temp/2017/073_SALES_DATA_NNME_17072017.DAT', '../data/temp/2017/005_SALES_DATA_NNME_20032017.DAT', '../data/temp/2017/010_SALES_DATA_NNME_06112017.DAT', '../data/temp/2017/124_SALES_DATA_NNME_17042017.DAT', '../data/temp/2017/069_SALES_DATA_NNME_01052017.DAT', '../data/temp/2017/528_SALES_DATA_NNME_18092017.DAT', '../data/temp/2017/055_SALES_DATA_NNME_12062017.DAT', '../data/temp/2017/152_SALES_DATA_NNME_07082017.DAT', '../data/temp/2017/116_SALES_DATA_NNME_13032017.DAT', '../data/temp/2017/205_SALES_DATA_NNME_02102017.DAT', '../data/temp/201

In [22]:
#load_sales_data("../data/temp/{0}".format(yr),-3)

### Data Mart Load

In [8]:
## History Load 
#for yr in range(2011,2018):
#    print(yr)
#    sale_datamart(str(yr))

In [14]:
# Clean up temp space
#! rm -r ../data/temp/*
#! rm -r ../data/sales/*/'*.csv'

In [31]:
! ls -l ../data/temp/

total 0
drwxr-xr-x 6874 jovyan users 219968 Jan 14 23:32 2018
drwxr-xr-x  136 jovyan users   4352 Jan 14 03:15 20181119
drwxr-xr-x  136 jovyan users   4352 Jan 14 03:15 20181126
drwxr-xr-x  135 jovyan users   4320 Jan 14 03:15 20181203
drwxr-xr-x   91 jovyan users   2912 Jan 14 22:39 20190107
drwxr-xr-x  121 jovyan users   3872 Jan 14 22:39 20190114
drwxr-xr-x  134 jovyan users   4288 Jan 31 22:39 20190121
drwxr-xr-x  128 jovyan users   4096 Feb 11 05:59 20190128
drwxr-xr-x  135 jovyan users   4320 Feb 11 05:59 20190204
drwxr-xr-x  135 jovyan users   4320 Feb 11 05:59 20190211
drwxr-xr-x  133 jovyan users   4256 Feb 19 00:13 20190218


In [30]:
import os
for file in os.listdir("../data/temp/"):
    if file.endswith(".DAT"):
        print(os.path.join("../data/temp/", file))